You've written a solid Pydantic v2 model using `@field_validator`. Let’s **analyze, explain, and execute step by step** to help you deeply understand what's happening — especially how `field_validator` works in this exact context.

---

## ✅ Code Overview

```python
from pydantic import BaseModel, EmailStr, AnyUrl, Field, field_validator
from typing import List, Dict, Optional, Annotated
```

### You're importing:

* `BaseModel`: base class for Pydantic models.
* `EmailStr`: enforces proper email format.
* `field_validator`: Pydantic v2 validator for individual fields.

---

## 🏥 Model: `Patient`

```python
class Patient(BaseModel):
    name: str
    email: EmailStr
    age: int
    weight: float
    married: bool
    allergies: List[str]
    contact_details: Dict[str, str]
```

### Fields

| Field             | Type             | Notes                          |
| ----------------- | ---------------- | ------------------------------ |
| `name`            | `str`            | Will be converted to uppercase |
| `email`           | `EmailStr`       | Also domain-restricted         |
| `age`             | `int`            | Validated to be 1–99           |
| `weight`          | `float`          | No extra validation            |
| `married`         | `bool`           | Coerces truthy/falsy           |
| `allergies`       | `List[str]`      | List of strings required       |
| `contact_details` | `Dict[str, str]` | E.g. phone/email/other         |

---

## 🔍 Field Validators in Action

### 1. Email domain validation

```python
@field_validator('email')
@classmethod
def email_validator(cls, value):
    valid_domains = ['hdfc.com', 'icici.com']
    domain_name = value.split('@')[-1]
    if domain_name not in valid_domains:
        raise ValueError('Not a valid domain')
    return value
```

* Works **after** EmailStr has validated format.
* Ensures domain is only `'hdfc.com'` or `'icici.com'`.

---

### 2. Name transformation to UPPERCASE

```python
@field_validator('name')
@classmethod
def transform_name(cls, value):
    return value.upper()
```

* Used for **transformation**, not validation.
* Converts input like `nitish` → `NITISH`.

---

### 3. Age validation in range (1–99)

```python
@field_validator('age', mode='after')
@classmethod
def validate_age(cls, value):
    if 0 < value < 100:
        return value
    else:
        raise ValueError('Age should be in between 0 and 100')
```

* Checks if parsed `int` is in valid range.

---

## 🧪 Example Input

```python
patient_info = {
    'name': 'nitish',
    'email': 'abc@icici.com',
    'age': '30',
    'weight': 75.2,
    'married': True,
    'allergies': ['pollen', 'dust'],
    'contact_details': {'phone': '2353462'}
}
```

* Even though `age` is passed as a **string**, Pydantic coerces it to `int`.
* `email` is validated for domain.
* `name` becomes uppercase: `NITISH`.

---

## 🔄 Output from `update_patient_data`

```python
def update_patient_data(patient: Patient):
    print(patient.name)         # NITISH
    print(patient.age)          # 30
    print(patient.allergies)    # ['pollen', 'dust']
    print(patient.married)      # True
    print('updated')
```

### Output:

```text
NITISH
30
['pollen', 'dust']
True
updated
```

---

## ❌ What if invalid input is passed?

### Example: Invalid domain

```python
Patient(**{..., 'email': 'abc@gmail.com'})
```

❌ Will raise:

```text
pydantic_core._pydantic_core.ValidationError: 1 validation error for Patient
email
  Not a valid domain (type=value_error)
```

### Example: Age > 100

```python
Patient(**{..., 'age': 150})
```

❌ Will raise:

```text
pydantic_core._pydantic_core.ValidationError: 1 validation error for Patient
age
  Age should be in between 0 and 100 (type=value_error)
```

---

## 🔁 Quick Notes

* You used `@classmethod` correctly.
* You could replace `@classmethod` with `@staticmethod` in Pydantic v2 – both work, but `@classmethod` is standard in validators.
* Pydantic v2 allows validators for **multiple fields** too via `@model_validator`.

---

## ✅ Conclusion

This is a **clean and production-ready Pydantic v2 model** using `field_validator`. You’ve used:

* Domain restriction on email,
* Transformational logic on name,
* Range validation on age.

Let me know if you want:

* Cross-field validation with `@model_validator`,
* Dynamic default values,
* Serialization (e.g. `.json()`),
* Or how to generate a FastAPI schema from this model.
